In [ ]:
import sys
sys.path.append("././quaos")
from math import gcd

from paulis import PauliSum, PauliString, Pauli
import numpy as np
from circuits import Circuit, Gate, SUM as CX, Hadamard as H, PHASE as S, SWAP
from hamiltonian import random_pauli_hamiltonian, cancel_pauli, symplectic_pauli_reduction, pauli_reduce

In [ ]:
p = PauliSum(['x1z0 x1z1'], dimensions=[2, 2])
p2 = PauliSum(['x1z1 x0z1'], dimensions=[2, 2])

for x0 in range(2):
    for z0 in range(2):
        for x1 in range(2):
            for z1 in range(2):
                q = PauliSum([f'x{x0}z{z0} x{x1}z{z1}'], dimensions=[2, 2])
                print((q * p).phases, (p * q).phases)



In [ ]:

# Pauli multiplication phase table over GF(2) symplectic vectors
# Represent each Pauli P as (x|z|phase) over GF(2) and integers mod 4
# phase in {0, 1, 2, 3} corresponds to {+1, +i, -1, -i}

# Map Pauli symplectic representation to human-readable labels
pauli_labels = {
    (0, 0): 'I',
    (1, 0): 'X',
    (0, 1): 'Z',
    (1, 1): 'Y'
}

def symplectic_commute(p1, p2):
    """Check if two Paulis commute based on symplectic product."""
    x1, z1 = p1[:len(p1)//2], p1[len(p1)//2:]
    x2, z2 = p2[:len(p2)//2], p2[len(p2)//2:]
    return (np.dot(x1, z2) + np.dot(x2, z1)) % 2

def apply_symplectic_cnot(p):
    """Apply 2-qubit CNOT symplectic matrix to Pauli (x|z)."""
    x = p[:2].copy()
    z = p[2:4].copy()
    x_new = np.array([x[0], x[0] ^ x[1]], dtype=int)
    z_new = np.array([z[0] ^ z[1], z[1]], dtype=int)
    return np.concatenate([x_new, z_new])

def track_phase_change(original_p, transformed_p, orig_phase):
    """
    Compute new phase (mod 4) using commutation rules:
    If conjugation anticommutes with original, flip sign (add 2 mod 4).
    """
    # Here, just test commutation between original and transformed
    if symplectic_commute(original_p, transformed_p) == 1:
        return (orig_phase + 2) % 4
    else:
        return orig_phase

for x0 in range(2):
    for x1 in range(2):
        for z0 in range(2):
            for z1 in range(2):

                # Example: input Pauli YI (iXZ) -> symplectic (1,0 | 1,0), phase 1 (i)
                pauli_symp = np.array([x0, x1, z0, z1])  # 
                phase = 1  # phase = 1

                # Step 1: Apply symplectic transformation from CNOT
                pauli_symp_cnot = apply_symplectic_cnot(pauli_symp)

                # Step 2: Recover new phase
                new_phase = track_phase_change(pauli_symp, pauli_symp_cnot, phase)

                print(pauli_symp, pauli_symp_cnot, phase, new_phase)